Table of contents
 * [초기 django 프로젝트 생성](#초기-django-프로젝트-생성)
 * [기본 django app 에 대해 migrate](#기본-django-app-에-대해-migrate)
 * [superuser 계정 생성](#superuser-계정-생성)
 * [최초의 blog 앱 생성](#최초의-blog-앱-생성)
  * [settings 의 INSTALLED_APPS 에 blog 앱 추가](#settings-의-INSTALLED_APPS-에-blog-앱-추가)

 * [blog 앱, Post 모델 생성](#blog-앱,-Post-모델-생성)
  * [마이그레이션 적용](#마이그레이션-적용)
  * [admin 에 Post 모델 등록](#admin-에-Post-모델-등록)

 * Post 목록보기/내용보기 뷰 구현
  * [blog 방문객들이 Post 를 조회할 수 있도록, Post 목록 뷰 작성](#blog-방문객들이-Post-를-조회할-수-있도록,-Post-목록-뷰-작성)
  * [blog 방문객들이 각 Post 의 내용을 조회할 수 있도록, Post Detail 뷰 작성](#blog-방문객들이-각-Post-의-내용을-조회할-수-있도록,-Post-Detail-뷰-작성)
  * [post index 템플릿에서 post detail 뷰로의 링크 추가](#blog-방문객들이-각-Post-의-내용을-조회할-수-있도록,-Post-Detail-뷰-작성)

 * [모델에 이미지를 저장해보자](#모델에-이미지를-저장해보자)
  * [이미지 저장 경로를 변경해보자.](#이미지-저장-경로를-변경해보자.)
  * [Post 모델이 저장이 될 때,이미지를 체크하여,최대 1024픽셀 이미지로 변경해보자.](#Post-모델이-저장이-될-때,-이미지를-체크하여,-최대-1024픽셀-이미지로-변경해보자.)

 * TODO
  * url reverse 의 3가지 방법
  * Pillow 를 이용한 이미지 처리
  * signals (pre_save, post_save)
  * ImageField 랜덤 파일명 지정
  * Template Loader
  * Static Files Finders
  * ngrok


## 초기 django 프로젝트 생성

In [ ]:
python manage.py startproject project3
cd project3

## 기본 django app 에 대해 migrate

In [ ]:
python manage.py migrate

기본 django app 에서는 이미 migrations 파일들이 생성되어 있으므로, makemigartions 이 따로 필요없습니다. migrate 만 하면 충분합니다.

## superuser 계정 생성

In [ ]:
python manage.py createsuperuser

생성된 계정으로, http://localhost:8000/admin/ 에 로그인이 가능합니다. :D

## 최초의 blog 앱 생성

In [ ]:
python manage.py startapp blog

### settings 의 INSTALLED_APPS 에 blog 앱 추가

In [ ]:
# project3/settings.py
INSTALLED_APPS = (
    # (중략) ...
    'blog',
)

INSTALLED_APPS : django 프로젝트 내 관리대상 app 목록

 * 여기에 포함되지 않으면 ...
  * 마이그레이션 대상에서 제외
  * 차후 배울 templates, static loader 대상에서도 제외
  * urls/view 라우팅은 project/urls.py 에서 직접 라우팅하기 때문에, INSTALLED_APPS 에 포함되지 않아도 가능함.

## blog 앱, Post 모델 생성

In [ ]:
# blog/models.py 파일
from django.db import models

class Post(models.Model):
    title = models.CharField(max_length=100)
    content = models.TextField()
    created_at = models.DateTimeField(auto_now_add=True)
    updated_at = models.DateTimeField(auto_now=True)

### 마이그레이션 적용

In [ ]:
python manage.py makemigrations
python manage.py migrate

### admin 에 Post 모델 등록

In [ ]:
# blog/admin.py 파일
from django.contrib import admin
from blog.models import Post

admin.site.register(Post)

이제 admin 페이지에서 Post 모델 관련 CRUD 작업을 할 수 있습니다. http://localhost:8000/admin/ 에서 Post 데이터를 10개 정도 생성해보세요.

## blog 방문객들이 Post 를 조회할 수 있도록, Post 목록 뷰 작성

In [ ]:
# project3/urls.py
from django.conf.urls import include, url
from django.contrib import admin

urlpatterns = [
    url(r'^admin/', include(admin.site.urls)),
    url(r'^blog/', include('blog.urls')),  ## 추가됨.
]

In [ ]:
# blog/urls.py
from django.conf.urls import url

urlpatterns = [
    url(r'^$', 'blog.views.index'),
]

In [ ]:
# blog/views.py
from django.shortcuts import render
from blog.models import Post

def index(request):
    post_list = Post.objects.all()
    return render(request, 'blog/index.html', {
        'post_list': post_list,
    })

```html
<!-- blog/templates/blog/index.html-->
<style>
td {
    border: 1px solid red;
    background-color: lightyellow;
}
</style>

<table>
    {% for post in post_list %}
    <tr>
        <td>{{ post.id }}</td>
        <td>{{ post.title }}</td>
        <td>{{ post.updated_at }}</td>
    </tr>
    {% endfor %}
</table>
```

이제 http://localhost:8000/blog/ 주소를 통해 post 목록을 확인할 수 있습니다.

## blog 방문객들이 각 Post 의 내용을 조회할 수 있도록, Post Detail 뷰 작성

In [ ]:
# blog/urls.py
from django.conf.urls import url

urlpatterns = [
    url(r'^$', 'blog.views.index'),
    url(r'^(?P<pk>\d+)/$', 'blog.views.detail'),  # 추가됨.
]

In [ ]:
# blog/views.py

def detail(request, pk):
    post = Post.objects.get(pk=pk)
    return render(request, 'blog/detail.html', {
        'post': post,
    })

```html
<!-- blog/templates/blog/detail.html -->

<h1>{{ post.title }}</h1>

{{ post.content }}

<hr/>
<a href="/blog/">글 목록</a>
```

이제 http://localhost:8000/blog/1/ 등의 주소를 통해 post 내용을 확인할 수 있습니다.

## post index 템플릿에서 post detail 뷰로의 링크 추가

```html
<!-- blog/templates/blog/index.html-->
<style>
td {
    border: 1px solid red;
    background-color: lightyellow;
}
</style>

<table>
    {% for post in post_list %}
    <tr>
        <td>{{ post.id }}</td>
        <!-- 링크 부분만 추가됨. -->
        <td><a href="/blog/{{ post.id }}/">{{ post.title }}</a></td>
        <td>{{ post.updated_at }}</td>
    </tr>
    {% endfor %}
</table>
```

이제 링크만 눌러도, 각 post 내용을 즉시 확인할 수 있습니다.

## 모델에 이미지를 저장해보자

In [ ]:
# project/settings.py

MEDIA_URL = '/media/'                          # 이미지의 URL 을 만들 때, prefix 로 쓰인다.
MEDIA_ROOT = os.path.join(BASE_DIR, 'media')   # 이미지가 저장될 경로

In [7]:
# blog/models.py

class Post(models.Model):
    # 중략
    image = models.ImageField(blank=True, null=True)

모델의 내역이 변경되었으므로, 마이그레이션을 수행해줍니다. 이제 관리자 페이지 (http://localhost:8000/admin/) 를 통해, Post 에 이미지를 등록할 수 있습니다.

이미지 등록은 되지만 이미지 링크를 열어보면 *404 Page not found* 예외가 발생합니다. 이는 MEDIA 파일들에 대한 뷰처리가 되어있지 않기 때문입니다.
 * 개발서버 (현재, settings.DEBUG=True) : 개발서버에서는 직접 MEDIA 파일들에 대한 뷰처리를 해줘야 합니다.
 * 실서비스 서버 (settings.DEBUG=False) : 실서비스 서버에서는 대개 django 이전에 nginx/apache 에서 MEDIA 파일들에 대해서 처리를 해주기 때문에, django 에서 뷰 처리를 해줄 필요가 없습니다.

In [ ]:
# project/urls.py
from django.conf import settings
from django.conf.urls import include, url
from django.contrib import admin

urlpatterns = [
    url(r'^admin/', include(admin.site.urls)),
    url(r'^blog/', include('blog.urls')),
]

if settings.DEBUG:
    urlpatterns += [
        url(r'^media/(?P<path>.*)$', 'django.views.static.serve', {
            'document_root': settings.MEDIA_ROOT,
        }),
    ]


이제 이미지 링크를 통해, 이미지가 잘 보여집니다.

### 이미지 저장 경로를 변경해보자.

ImageField 에 upload_to 인자를 적용해서
 * **upload_to='mypython'** 로 지정 시 (저장 디렉토리 지정)
  * settings.MEDIA_ROOT 디렉토리에 "mypython/업로드된파일명" 경로로 저장이 됩니다.
 * **upload_to='mypython/mydjango/myhello'** 로 지정 시 (저장 디렉토리 지정)
  * settings.MEDIA_ROOT 디렉토리에 "mypython/mydjango/myhello/업로드된파일명" 경로로 저장이 됩니다.
 * **upload_to='%Y/%m/%d'** 로 지정 시 (저장 디렉토리 지정)
  * settings.MEDIA_ROOT 디렉토리에 "2015/07/30/업로드된파일명" 경로로 저장이 됩니다.
  * '%Y/%m/%d' 부분은 각각 년/월/일로 변환됩니다.
 * **upload_to=함수** 로 지정 시 (저장 파일경로 지정)
  * 매번 함수의 리턴값을 경로로 받아서 처리합니다.
  * 함수에는 2개의 인자가 넘겨집니다. (모델 인스턴스, 원 파일명)
  * 함수에서 "mypython/mydjango/myhello/my.jpg" 리턴을 하면, settings.MEDIA_ROOT 디렉토리에 "mypython/mydjango/myhello/my.jpg" 로 저장이 됩니다.

아래와 같이 지정하면, settings.MEDIA_ROOT 디렉토리에 현재 년/월/일 디렉토리에, 업로드된 파일명을 유지한 채로 저장이 됩니다.

In [ ]:
# blog/models.py

class Post(models.Model):
    # 중략
    image = models.ImageField(upload_to='%Y/%m/%d', blank=True, null=True)

업로드된 파일명을 랜덤파일명으로 지정코자 할 때에는 필히, upload_to 에 함수로 인자를 지정해줘야 합니다. 그것의 가장 간단한 형태로서, 다음과 같이 simple_random_name_upload_to 를 지정해볼 수 있겠습니다.

In [9]:
import os
from uuid import uuid4

# 이미지 랜덤 파일명 지정의 가장 단순한 형태
def simple_random_name_upload_to(model_instance, filename):
    extension = os.path.splitext(filename)[-1].lower()
    return uuid4().hex + extension

### Post 모델이 저장이 될 때, 이미지를 체크하여, 최대 1024픽셀 이미지로 변경해보자.

In [11]:
# TODO